<h1>Prepare Data</h1>

In [ ]:
FILE_ORIGIN = "./student_data.csv"
FILE_DESTINATION = "./student_data_final.csv"
TESTS_AND_CHAPTERS_FOR_SUBJECTS =  [
        ("Test1", ["English", "Maths"], ["A,B", "C,D"]),
        ("Test2", ["English", "Maths"], ["B,C", "D,F"])
    ]

<h1>Add topics to csv Data</h1>

In [ ]:
import csv
from typing import List, Tuple, Dict

def add_topics_to_csv():
    """
    - input_csv_path, output_csv_path: file paths.
    - tests: a list of (test_name, subjects, topic_values), e.g.:
        [
          ("Test1", ["English","Maths"], ["A,B","C,D"]),
          ("Test2", ["English","Maths"], ["B,C","E,F"])
        ]
    
    This will add columns:
      English Topics Test1, Maths Topics Test1,
      English Topics Test2, Maths Topics Test2,
      English All Topics,  Maths All Topics
    """
    # 1) Validate
    for test_name, subjects, chapters in TESTS_AND_CHAPTERS_FOR_SUBJECTS:
        if len(subjects) != len(chapters):
            raise ValueError(f"subjects vs topic_values length mismatch in {test_name}")

    # 2) Collect all unique subjects and append it to all_subjects
    all_subjects: List[str] = []
    for _, subjects, _ in TESTS_AND_CHAPTERS_FOR_SUBJECTS:
        for subj in subjects:
            if subj not in all_subjects:
                all_subjects.append(subj)

    # 3) Build per-test lookup maps
    test_topic_maps: Dict[str, Dict[str, str]] = {
        test_name: dict(zip(subjects, chapters))
        for test_name, subjects, chapters in TESTS_AND_CHAPTERS_FOR_SUBJECTS
    }

    # 4) Open I/O
    with open(FILE_ORIGIN, newline="", encoding="utf-8") as fin, \
         open(FILE_DESTINATION, "w", newline="", encoding="utf-8") as fout:

        reader = csv.DictReader(fin)
        # a) build the new header
        extra_cols: List[str] = []
        for test_name in test_topic_maps:
            for subj in all_subjects:
                if subj in test_topic_maps[test_name]:
                    extra_cols.append(f"{subj} Topics {test_name}")
        for subj in all_subjects:
            extra_cols.append(f"{subj} All Topics")

        writer = csv.DictWriter(fout, fieldnames=reader.fieldnames + extra_cols)
        writer.writeheader()

        # 5) Process each row
        for row in reader:
            # per-test columns
            for test_name, topics_map in test_topic_maps.items():
                for subj, topics_str in topics_map.items():
                    row[f"{subj} Topics {test_name}"] = topics_str

            # aggregated union columns
            for subj in all_subjects:
                all_toks: List[str] = []
                for topics_map in test_topic_maps.values():
                    if subj in topics_map:
                        # split on comma, strip whitespace
                        all_toks.extend([tok.strip() for tok in topics_map[subj].split(",")])
                # dedupe & sort (optional)
                unique = sorted(set(tok for tok in all_toks if tok))
                row[f"{subj} All Topics"] = ", ".join(unique)

            writer.writerow(row)


In [ ]:
import re
import pandas as pd

def validate_data(df: pd.DataFrame, test_mark_cols=None) -> pd.DataFrame:
    """
    Validate and normalize the student scores dataframe in-place.
    Preserves 'AB' values for absent students while ensuring data quality.

    Expected columns in the new CSV:
      - 'Student Names'
      - Any number of '<Subject>_Test<N>' columns (e.g. 'English_Test1', 'Maths_Test2', …)
      - 'Attendance'                      # numeric
      - "Teacher's Remarks"
      - Topic columns: any column whose name (case-insensitive) contains 'topics'
        (e.g. 'English Topics Test1', 'Maths All Topics', etc.)

    Returns:
        The validated and normalized dataframe
    """
    df = df.copy()

    # 0. Trim whitespace from all object-dtype columns
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].str.strip()

    # 1. Normalize topic columns (ensure “A, B, C” formatting)
    topic_cols = [c for c in df.columns if re.search(r'topics', c, re.IGNORECASE)]
    for col in topic_cols:
        df[col] = df[col].apply(
            lambda x: ', '.join(p.strip() for p in x.split(',')) if isinstance(x, str) else x
        )

    # 2. Check missing student names
    if df['Student Names'].isna().any():
        missing = df[df['Student Names'].isna()].index.tolist()
        print(f"Missing Student Names in rows: {missing}")

    # 3. Identify all mark columns: either passed in, or auto-detect *_Test# columns
    if test_mark_cols is None:
        test_mark_cols = [c for c in df.columns if re.match(r'.+_Test\d+', c)]
    # Make sure they exist
    missing_marks = [c for c in test_mark_cols if c not in df.columns]
    if missing_marks:
        raise KeyError(f"Expected mark columns not found: {missing_marks}")

    # 4. Validate each test mark column, preserving 'AB'
    for col in test_mark_cols:
        # a) Empty → 'AB'
        empty_mask = df[col].isna() | (df[col] == '')
        if empty_mask.any():
            for idx in df[empty_mask].index:
                name = df.at[idx, 'Student Names'] or '<Unknown>'
                print(f"Missing {col} for {name}; marking as absent ('AB')")
            df.loc[empty_mask, col] = 'AB'

        # b) Standardize 'AB' (case-insensitive)
        is_ab = df[col].astype(str).str.upper().str.strip() == 'AB'
        df.loc[is_ab, col] = 'AB'

        # c) Convert anything else to numeric (int if whole, float otherwise); non-convertible → 'AB'
        for idx in df.index:
            if df.at[idx, col] == 'AB':
                continue
            val = df.at[idx, col]
            try:
                num = float(val)
                df.at[idx, col] = int(num) if num.is_integer() else num
            except Exception:
                name = df.at[idx, 'Student Names'] or '<Unknown>'
                print(f"Invalid {col} value '{val}' for {name}; marking as absent")
                df.at[idx, col] = 'AB'

    # 5. Attendance: fill missing → 0, then force int
    if 'Attendance' not in df.columns:
        raise KeyError("Expected column 'Attendance' not found")
    if df['Attendance'].isna().any():
        for idx in df[df['Attendance'].isna()].index:
            name = df.at[idx, 'Student Names'] or '<Unknown>'
            print(f"Attendance missing for {name}; setting to 0")
        df['Attendance'] = df['Attendance'].fillna(0)
    df['Attendance'] = pd.to_numeric(df['Attendance'], errors='coerce').fillna(0).astype(int)

    # 6. Summary
    print(f"\nNumber of students loaded: {len(df)}")
    absent_summary = {col: (df[col] == 'AB').sum() for col in test_mark_cols if (df[col] == 'AB').any()}
    if absent_summary:
        print("\nStudents marked as absent:")
        for col, count in absent_summary.items():
            print(f"  {col}: {count}")

    # 7. Sample first 5 rows
    print(df.head(5).to_string(index=False))

    return df


In [ ]:
add_topics_to_csv()